In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [2]:
def print_to_file(value, name):
    print(name, value)
    with open(name, "w") as file:
        file.write(str(value))

# Примеры задач анализа текстов. Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

## Задание

1. Загрузите датасет. Описание датасета можно посмотреть [здесь](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection).

2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [3]:
df = pd.read_csv('data/SMSSpamCollection.txt', sep = '\t', header=None, names = ('status', 'text'))
df.head()

,status,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [4]:
X = df.text
y = df.status
y = y.replace('spam', 1)
y = y.replace('ham', 0)

4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.
5. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [5]:
#vectorizer = CountVectorizer()
#X = vectorizer.fit_transform(X)
#clf = LogisticRegression(random_state=2).fit(X, y)
#print(cross_val_score(clf, X, y, cv=10, scoring='f1').mean())

In [6]:
def text_classifier(vectorizer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("classifier", classifier)]
        )

In [7]:
pipeline = text_classifier(CountVectorizer(), LogisticRegression())
score_log = cross_val_score(pipeline, X, y, cv=10, scoring='f1').mean()

In [8]:
print_to_file(round(score_log, 1),'1.txt')

1.txt 0.9


6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:


```
"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"
```

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [9]:
test_texts = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
              "FreeMsg: Txt: claim your reward of 3 hours talk time",
              "Have you visited the last lecture on physics?",
              "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
              "Only 99$"]

In [10]:
pipeline.fit(X, y)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
             

In [11]:
predicted = ' '.join(map(str, pipeline.predict(test_texts)))

In [12]:
print_to_file(predicted,'2.txt')

2.txt 1 1 0 0 0


7. Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [13]:
ngram_ranges = [(2,2),(3,3),(1,3)]
scores = []
for ngram_range in ngram_ranges:
    pipeline = text_classifier(CountVectorizer(ngram_range = ngram_range),
                               LogisticRegression())
    score = cross_val_score(pipeline, X, y, cv=10, scoring='f1').mean()
    scores.append(round(score, 2))

In [14]:
scores_without_commas = ' '.join(map(str, scores))
print_to_file(scores_without_commas,'3.txt')

3.txt 0.82 0.72 0.92


8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [15]:
ngram_ranges = [(2,2),(3,3),(1,3)]
scores = []
for ngram_range in ngram_ranges:
    pipeline = text_classifier(CountVectorizer(ngram_range = ngram_range),
                               MultinomialNB())
    score = cross_val_score(pipeline, X, y, cv=10, scoring='f1').mean()
    scores.append(round(score, 2))

In [16]:
scores_without_commas = ' '.join(map(str, scores))
print_to_file(scores_without_commas,'4.txt')

4.txt 0.93 0.87 0.95


Грейдер не принимает ответы наивного байесовского классификатора через Pipeline. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

In [17]:
ngram_ranges = [(2,2),(3,3),(1,3)]
scores = []
for ngram_range in ngram_ranges:
    vectorizer = CountVectorizer(ngram_range = ngram_range)
    X_vec = vectorizer.fit_transform(X)
    clf = MultinomialNB().fit(X_vec, y)
    score = cross_val_score(clf, X_vec, y, cv=10, scoring='f1').mean()
    scores.append(round(score, 2))

In [18]:
scores_without_commas = ' '.join(map(str, scores))
print_to_file(scores_without_commas,'4.txt')

4.txt 0.65 0.38 0.89


9. Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [19]:
pipeline = text_classifier(TfidfVectorizer(), LogisticRegression())
score_tfidf = cross_val_score(pipeline, X, y, cv=10, scoring='f1').mean()
print(score_tfidf)

0.8776671468367306


In [20]:
diff_answer = 0
diff = score_tfidf - score_log
if abs(diff) <= 0.01:
    diff_answer = 0
elif diff > 0:
    diff_answer = 1
else:
    diff_answer = -1

In [21]:
print_to_file(diff_answer,'5.txt')

5.txt -1
